# vTTS + CosyVoice3 테스트

## 설치: 한 줄!

```bash
pip install "vtts[cosyvoice] @ git+https://github.com/bellkjtt/vTTS.git"
```

- CosyVoice 코드가 **vTTS에 내장**
- 별도 클론 불필요!
- 모델만 HuggingFace에서 자동 다운로드

**⚠️ Kaggle GPU 필수**: Settings → Accelerator → GPU T4 x2

## Step 1: 설치 (한 셀에서 완료!)

In [ ]:
%%bash
# vTTS + CosyVoice 한 줄 설치!
pip install "vtts[cosyvoice] @ git+https://github.com/bellkjtt/vTTS.git"

echo ""
echo "✅ 설치 완료!"
echo "⚠️ 이제 '런타임 > 세션 다시 시작'을 클릭하세요!"

## Step 2: 커널 재시작 후 버전 확인

In [ ]:
# 버전 확인 (커널 재시작 후 실행!)
import numpy as np
import torch
import whisper

print("🔍 버전 확인")
print(f"numpy: {np.__version__}")
print(f"torch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
print(f"whisper: OK")

# CosyVoice 내장 모듈 확인
from vtts.engines._cosyvoice.cli.cosyvoice import AutoModel
print("\n✅ CosyVoice 내장 모듈: OK")

## 3. 참조 오디오 생성 (Supertonic)

In [ ]:
# Supertonic으로 참조 오디오 생성
from pathlib import Path
from vtts.engines.supertonic import SupertonicEngine
from vtts.engines.base import EngineTTSRequest

print("🎙️ 참조 오디오 생성 중...\n")

engine = SupertonicEngine(model_id="Supertone/supertonic-2", device="cuda")
engine.load_model()

ref_dir = Path("./reference_audio")
ref_dir.mkdir(exist_ok=True)

# 한국어
ko_text = "안녕하세요, 저는 음성 클로닝을 위한 참조 음성입니다."
ko_path = ref_dir / "reference_ko.wav"
ko_request = EngineTTSRequest(text=ko_text, voice="F1", language="ko")
ko_audio = engine.synthesize(ko_request)
engine.save_audio(ko_audio, str(ko_path))
print(f"✅ 한국어: {ko_path}")

# 영어
en_text = "Hello, this is a reference audio for voice cloning."
en_path = ref_dir / "reference_en.wav"
en_request = EngineTTSRequest(text=en_text, voice="F1", language="en")
en_audio = engine.synthesize(en_request)
engine.save_audio(en_audio, str(en_path))
print(f"✅ 영어: {en_path}")

engine.unload_model()
print("\n✅ 완료!")

## 4. vTTS 서버 시작

In [ ]:
# vTTS 서버 시작
import subprocess
import time
import httpx

print("🚀 vTTS 서버 시작 중...\n")

server_process = subprocess.Popen(
    ["vtts", "serve", "FunAudioLLM/Fun-CosyVoice3-0.5B-2512",
     "--host", "0.0.0.0", "--port", "8000", "--device", "cuda"],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1
)

print("서버 시작 대기 중 (최대 60초)...")
for i in range(60):
    try:
        response = httpx.get("http://localhost:8000/health", timeout=1.0)
        if response.status_code == 200:
            print(f"\n✅ 서버 준비 완료! ({i+1}초)")
            print("   API: http://localhost:8000/v1")
            break
    except:
        pass
    time.sleep(1)
    if i % 10 == 0:
        print(f"   대기 중... {i+1}s")
else:
    print("\n❌ 시간 초과!")

## 5. CosyVoice Zero-Shot TTS 테스트

In [ ]:
# Zero-Shot TTS 테스트
from vtts import VTTSClient
import IPython.display as ipd
from pathlib import Path

client = VTTSClient(base_url="http://localhost:8000")

print("🎙️ CosyVoice Zero-Shot TTS\n" + "="*60)

# 한국어
print("\n[1] 한국어")
ko_path = Path("./reference_audio/reference_ko.wav").absolute()
ko_audio = client.tts(
    text="이것은 코지보이스를 사용한 한국어 음성 클로닝 테스트입니다.",
    model="FunAudioLLM/Fun-CosyVoice3-0.5B-2512",
    language="ko",
    reference_audio=str(ko_path),
    reference_text="안녕하세요, 저는 음성 클로닝을 위한 참조 음성입니다."
)
print(f"✅ 완료! Duration: {len(ko_audio.audio)/ko_audio.sample_rate:.2f}s")
ko_audio.save("cosyvoice_korean.wav")
display(ipd.Audio(ko_audio.audio, rate=ko_audio.sample_rate))

# 영어
print("\n[2] 영어")
en_path = Path("./reference_audio/reference_en.wav").absolute()
en_audio = client.tts(
    text="This is an English voice cloning test using CosyVoice.",
    model="FunAudioLLM/Fun-CosyVoice3-0.5B-2512",
    language="en",
    reference_audio=str(en_path),
    reference_text="Hello, this is a reference audio for voice cloning."
)
print(f"✅ 완료! Duration: {len(en_audio.audio)/en_audio.sample_rate:.2f}s")
en_audio.save("cosyvoice_english.wav")
display(ipd.Audio(en_audio.audio, rate=en_audio.sample_rate))

print("\n" + "="*60)
print("✅ 테스트 완료!")

## 6. 정리 (서버 종료)

In [ ]:
# 서버 종료
print("🛑 서버 종료 중...")
server_process.terminate()
server_process.wait(timeout=5)
print("✅ 서버 종료 완료!")